In [1]:
%load_ext autoreload
%autoreload 2
from icu_pipeline.pipeline import (
    Pipeline,
    DataSource,
    SourceMapperConfiguration,
)
from icu_pipeline.mapper.sink.file import CSVFileSinkMapper
from pathlib import Path
from dotenv import load_dotenv
import os


load_dotenv()

POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
MIMIC_DB = os.getenv("MIMIC_DB")
AMDS_DB = os.getenv("AMDS_DB")
EICU_DB = os.getenv("EICU_DB")

/home/paul/projects/ukm/ICUPipeline/venv/lib64/python3.12/site-packages/pydantic/_internal/_fields.py:184: UserWarning: Field name "schema" shadows an attribute in parent "BaseModel"; 
  warnings.warn(


In [2]:
configs = {
    DataSource.MIMIC: SourceMapperConfiguration(
        f"postgresql+psycopg://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{MIMIC_DB}",
        limit=1234
    ),
}

pipeline = Pipeline(
    source_configs=configs,
    # sink_mapper=CSVFileSinkMapper(Path("./out"))
)

concepts = ["HeartRate"]

c = pipeline._load_concepts(concepts)
for a in c[0].__dict__.items():
    print(a)


('_concept_config', ConceptConfig(name='HeartRate', description='Heart rate is the speed of the heartbeat measured by the number of contractions of the heart per minute (bpm).', identifiers={<ConceptCoding.SNOMED: 'snomed'>: '364075005', <ConceptCoding.LOINC: 'loinc'>: '8867-4'}, unit='bpm', schema='FHIRObservation', mapper=[MapperConfig(klass='MimicObservationMapper', source='mimic', unit='bpm', params={'schema': 'mimiciv_icu', 'table': 'chartevents', 'constraints': {'itemid': '220045'}})]))
('_source_configs', {<DataSource.MIMIC: 'mimic'>: SourceMapperConfiguration(connection='postgresql+psycopg://paul:@localhost/mimiciv', chunksize=10000, limit=1234)})
('_concept_coding', <ConceptCoding.SNOMED: 'snomed'>)
('_fhir_schema', FHIRObservation)


In [3]:
result = pipeline.transform(concepts)
result

{'HeartRate': <generator object Concept.map at 0x7fc33b1e4040>}

In [4]:
d = next(result["HeartRate"])
d


            SELECT "subject_id" AS "patient_id", "charttime" AS "timestamp", "valuenum" AS "value", "valueuom" AS "unit"
            FROM "mimiciv_icu"."chartevents"
            WHERE "itemid" = '220045'
            LIMIT 1234
        


,subject,effective_date_time,value_quantity,code
0,"{'reference': '10000032', 'type': 'mimic-patie...",2180-07-23 22:00:00+00:00,"{'value': 94.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
1,"{'reference': '10000032', 'type': 'mimic-patie...",2180-07-23 19:00:00+00:00,"{'value': 97.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
2,"{'reference': '10000032', 'type': 'mimic-patie...",2180-07-23 20:00:00+00:00,"{'value': 100.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
3,"{'reference': '10000032', 'type': 'mimic-patie...",2180-07-23 21:00:00+00:00,"{'value': 94.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
4,"{'reference': '10000032', 'type': 'mimic-patie...",2180-07-23 14:12:00+00:00,"{'value': 91.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
...,...,...,...,...
1229,"{'reference': '10002428', 'type': 'mimic-patie...",2156-04-14 06:00:00+00:00,"{'value': 99.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
1230,"{'reference': '10002428', 'type': 'mimic-patie...",2156-04-12 17:26:00+00:00,"{'value': 124.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
1231,"{'reference': '10002428', 'type': 'mimic-patie...",2156-04-12 17:30:00+00:00,"{'value': 120.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
1232,"{'reference': '10002428', 'type': 'mimic-patie...",2156-04-12 17:45:00+00:00,"{'value': 134.0, 'unit': 'bpm'}","{'coding': {'code': '364075005', 'system': 'sn..."
